# モジュールの用意

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from inspect import signature
from selenium.webdriver.chrome.options import Options
import time
import csv
import re

# タグ一覧から記事数上位500件を取得

In [2]:
def eraseVersion(w):
    w = re.sub('(\d+(?:\.\d+)?)$', '', w)
    return w

In [3]:
with open("./qiita_tags.csv", "r") as f:
    tags = list(set([i for i in map(eraseVersion, f.read().split(","))][:500]))[1:]
print(tags)

['Groovy', 'mastodon', 'MacOSX', 'Bootstrap', 'RPA', 'GitHub', '初心者', 'DDD', 'プログラミング', 'FreeBSD', 'postfix', 'Yum', 'ネットワーク', 'JavaScript', 'Chainer', 'babel', 'TravisCI', 'Pepper', 'test', 'Phoenix', 'CloudWatch', 'OSS', 'データベース', 'SORACOM', 'archLinux', 'npm', '設計', 'websocket', 'mruby', 'bundler', 'フロントエンド', 'Zsh', 'Safari', 'iTerm', 'gas', 'coding', 'Design', 'UNIX', 'kernel', 'NLP', 'Alexa', 'ESP', 'ubuntu', 'Heroku', 'vue-cli', 'Railsチュートリアル', 'Bluemix', 'ionic', 'GoogleAppEngine', 'ゲーム制作', 'APIGateway', 'ECMAScript', '資格', 'laravel', 'HTML', 'Flask', 'ターミナル', 'Perl', 'devise', '数学', 'macos', 'Realm', 'データ分析', '勉強会', 'Git', 'アジャイル', 'Unity', 'Microsoft', 'borgWarp', 'GoogleMapsAPI', 'nuxt.js', 'awk', 'jsdo', '機械学習', 'LINQ', 'dockerfile', 'GIS', 'RxSwift', 'firefox', 'C#', 'Cygwin', 'データサイエンス', 'IAM', 'Ruby', '自然言語処理', 'WSL', 'VisualStudio', 'brew', 'Capybara', 'api', 'error', 'serverless', 'hadoop', '最適化', 'SourceTree', 'mariadb', 'matlab', 'HTTPS', 'difference', '初心者向け', 'シェルスク

# Qiita検索から上位・下位判定

In [4]:
def tagSearch(w, page="1"):
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Chrome(chrome_options=options)
        driver.get('https://qiita.com/search?page='+ page +'&sort=like&q=tag%3A' + w)
        data = driver.page_source
        html = BeautifulSoup(data)
        driver.close()
        driver.quit()
        return html

In [9]:
def check(w1, w2):
    w1 = w1.lower()
    w2 = w2.lower()
    res = []
    score = 0
    for p in range(1, 6):
        
        # w1で検索した際にw2が含まれていたら1 それ以外は0
        for i in tagSearch(w1, page=str(p)).find_all(class_="tagList"):
            res += [[eraseVersion(j.text.lower())==w2 for j in i.find_all("a")].count(True)]
        time.sleep(0.1)
        
        # 検索結果が存在しない場合スコア0を返す
        if len(res) == 0:
            return 0
        
        # w1で検索した時にw2が含まれる割合
        score += ((len(res)-res.count(0))+1e-7)/(len(res)+1e-7)
        
        # 1つのページの出現率が30%以下ならスコア0を返す
        if score/p < 0.3:
            return 0
        
    return round(score/p, 3)

In [10]:
def comparisonCheck(w1, w2, boundary=0.6):
    w1_score = check(w1, w2)
    w1, w2 = w2, w1
    w2_score = check(w1, w2)
    w1, w2 = w2, w1
    
    print(w1, "score:", w1_score)
    print(w2, "score:", w2_score)
    if (w1_score < boundary and w2_score < boundary) or (abs(w1_score-w2_score) < 0.1):
        print(w1, "and", w2, "are irrelevant")
        return {}
    else:
        if w1_score < w2_score:
            print(">>", w1, "is easy!")
            return {"easy": w1, "hard": w2, "score": max(w1_score, w2_score)}
        else:
            print(">>", w2, "is easy!")
            return {"easy": w2, "hard": w1, "score": max(w1_score, w2_score)}    

In [92]:
comparisonCheck("JavaScript", "HTML")

JavaScript score: 0
HTML score: 0.547
JavaScript and HTML are irrelevant


{}

In [12]:
results = []
start = 0

In [13]:
len(tags)

478

In [14]:
# 開始場所
s = start
for i, tag in enumerate(tags[start:len(tags)]):
    print("["+str(s+i)+"]:", tag)
    w1 = "Python"
    w2 = tag
    
    try:
        res = comparisonCheck(w1, w2)
        start += 1
    except:
        break
    
    if res:
        results.append(str(res))
    print()
    
with open("easy_hard.csv", "w") as f:
    f.write(",".join(results))
print(res)
print(start)

[0]: Groovy
Python score: 0
Groovy score: 0
Python and Groovy are irrelevant

[1]: mastodon
Python score: 0
mastodon score: 0
Python and mastodon are irrelevant

[2]: MacOSX
Python score: 0
MacOSX score: 0
Python and MacOSX are irrelevant

[3]: Bootstrap
Python score: 0
Bootstrap score: 0
Python and Bootstrap are irrelevant

[4]: RPA
Python score: 0
RPA score: 0
Python and RPA are irrelevant

[5]: GitHub
Python score: 0
GitHub score: 0
Python and GitHub are irrelevant

[6]: 初心者
Python score: 0
初心者 score: 0
Python and 初心者 are irrelevant

[7]: DDD
Python score: 0
DDD score: 0
Python and DDD are irrelevant

[8]: プログラミング
Python score: 0
プログラミング score: 0
Python and プログラミング are irrelevant

[9]: FreeBSD
Python score: 0
FreeBSD score: 0
Python and FreeBSD are irrelevant

[10]: postfix
Python score: 0
postfix score: 0
Python and postfix are irrelevant

[11]: Yum
Python score: 0
Yum score: 0
Python and Yum are irrelevant

[12]: ネットワーク
Python score: 0
ネットワーク score: 0
Python and ネットワーク are irrelev

Python score: 0
Qiita score: 0
Python and Qiita are irrelevant

[106]: デザインパターン
Python score: 0
デザインパターン score: 0
Python and デザインパターン are irrelevant

[107]: ExcelVBA
Python score: 0
ExcelVBA score: 0
Python and ExcelVBA are irrelevant

[108]: Elm
Python score: 0
Elm score: 0
Python and Elm are irrelevant

[109]: RubyMine
Python score: 0
RubyMine score: 0
Python and RubyMine are irrelevant

[110]: pip
Python score: 0
pip score: 0.942
>> Python is easy!

[111]: twilio
Python score: 0
twilio score: 0
Python and twilio are irrelevant

[112]: VisualStudioCode
Python score: 0
VisualStudioCode score: 0
Python and VisualStudioCode are irrelevant

[113]: Chrome
Python score: 0
Chrome score: 0
Python and Chrome are irrelevant

[114]: Laravel
Python score: 0
Laravel score: 0
Python and Laravel are irrelevant

[115]: 深層学習
Python score: 0
深層学習 score: 0.349
Python and 深層学習 are irrelevant

[116]: Linux
Python score: 0
Linux score: 0
Python and Linux are irrelevant

[117]: MachineLearning
Python score

Python score: 0
開発環境 score: 0
Python and 開発環境 are irrelevant

[207]: ImageMagick
Python score: 0
ImageMagick score: 0
Python and ImageMagick are irrelevant

[208]: shell
Python score: 0
shell score: 0
Python and shell are irrelevant

[209]: TeX
Python score: 0
TeX score: 0
Python and TeX are irrelevant

[210]: コマンド
Python score: 0
コマンド score: 0
Python and コマンド are irrelevant

[211]: ATOM
Python score: 0
ATOM score: 0
Python and ATOM are irrelevant

[212]: Subversion
Python score: 0
Subversion score: 0
Python and Subversion are irrelevant

[213]: chef
Python score: 0
chef score: 0
Python and chef are irrelevant

[214]: raspbian
Python score: 0
raspbian score: 0
Python and raspbian are irrelevant

[215]: bot
Python score: 0
bot score: 0
Python and bot are irrelevant

[216]: cli
Python score: 0
cli score: 0
Python and cli are irrelevant

[217]: Unity3D
Python score: 0
Unity3D score: 0
Python and Unity3D are irrelevant

[218]: Vagrant
Python score: 0
Vagrant score: 0
Python and Vagrant are

Python score: 0
FuelPHP score: 0
Python and FuelPHP are irrelevant

[309]: PhpStorm
Python score: 0
PhpStorm score: 0
Python and PhpStorm are irrelevant

[310]: Cisco
Python score: 0
Cisco score: 0
Python and Cisco are irrelevant

[311]: Keras
Python score: 0
Keras score: 0.798
>> Python is easy!

[312]: 関数型プログラミング
Python score: 0
関数型プログラミング score: 0
Python and 関数型プログラミング are irrelevant

[313]: Go
Python score: 0
Go score: 0
Python and Go are irrelevant

[314]: Pycharm
Python score: 0
Pycharm score: 0.71
>> Python is easy!

[315]: ASP.NET
Python score: 0
ASP.NET score: 0
Python and ASP.NET are irrelevant

[316]: ポエム
Python score: 0
ポエム score: 0
Python and ポエム are irrelevant

[317]: LINEmessagingAPI
Python score: 0
LINEmessagingAPI score: 0.324
Python and LINEmessagingAPI are irrelevant

[318]: analytics
Python score: 0
analytics score: 0
Python and analytics are irrelevant

[319]: zabbix
Python score: 0
zabbix score: 0
Python and zabbix are irrelevant

[320]: openFrameworks
Python scor

Python score: 0
AtCoder score: 0.391
Python and AtCoder are irrelevant

[411]: YouTube
Python score: 0
YouTube score: 0
Python and YouTube are irrelevant

[412]: Ubuntu
Python score: 0
Ubuntu score: 0
Python and Ubuntu are irrelevant

[413]: oracle
Python score: 0
oracle score: 0
Python and oracle are irrelevant

[414]: redux
Python score: 0
redux score: 0
Python and redux are irrelevant

[415]: JUnit
Python score: 0
JUnit score: 0
Python and JUnit are irrelevant

[416]: ブロックチェーン
Python score: 0
ブロックチェーン score: 0
Python and ブロックチェーン are irrelevant

[417]: cakephp
Python score: 0
cakephp score: 0
Python and cakephp are irrelevant

[418]: Twitter
Python score: 0
Twitter score: 0
Python and Twitter are irrelevant

[419]: HoloLens
Python score: 0
HoloLens score: 0
Python and HoloLens are irrelevant

[420]: テスト
Python score: 0
テスト score: 0
Python and テスト are irrelevant

[421]: DynamoDB
Python score: 0
DynamoDB score: 0
Python and DynamoDB are irrelevant

[422]: 数値計算
Python score: 0
数値計算 sco

In [15]:
start

478

In [16]:
results

["{'easy': 'Python', 'score': 0.781, 'hard': 'Chainer'}",
 "{'easy': 'Python', 'score': 0.893, 'hard': 'Flask'}",
 "{'easy': 'Python', 'score': 0.688, 'hard': '機械学習'}",
 "{'easy': 'Python', 'score': 0.808, 'hard': '自然言語処理'}",
 "{'easy': 'Python', 'score': 0.942, 'hard': 'pip'}",
 "{'easy': 'Python', 'score': 0.661, 'hard': 'mecab'}",
 "{'easy': 'Python', 'score': 1.0, 'hard': 'numpy'}",
 "{'easy': 'Python', 'score': 0.942, 'hard': 'Anaconda'}",
 "{'easy': 'Python', 'score': 0.949, 'hard': 'pandas'}",
 "{'easy': 'Python', 'score': 0.603, 'hard': 'DeepLearning'}",
 "{'easy': 'Python', 'score': 0.651, 'hard': '統計学'}",
 "{'easy': 'Python', 'score': 0.983, 'hard': 'scikit-learn'}",
 "{'easy': 'Python', 'score': 0.949, 'hard': 'Jupyter'}",
 "{'easy': 'Python', 'score': 0.909, 'hard': 'pyenv'}",
 "{'easy': 'Python', 'score': 0.798, 'hard': 'Keras'}",
 "{'easy': 'Python', 'score': 0.71, 'hard': 'Pycharm'}",
 "{'easy': 'Python', 'score': 1.0, 'hard': 'matplotlib'}",
 "{'easy': 'Python', 'score'

In [17]:
res = [eval(r) for r in results]

In [18]:
sorted(res, key=lambda x:x['score'], reverse=True)

[{'easy': 'Python', 'hard': 'numpy', 'score': 1.0},
 {'easy': 'Python', 'hard': 'matplotlib', 'score': 1.0},
 {'easy': 'Python', 'hard': 'scikit-learn', 'score': 0.983},
 {'easy': 'Python', 'hard': 'pandas', 'score': 0.949},
 {'easy': 'Python', 'hard': 'Jupyter', 'score': 0.949},
 {'easy': 'Python', 'hard': 'pip', 'score': 0.942},
 {'easy': 'Python', 'hard': 'Anaconda', 'score': 0.942},
 {'easy': 'Python', 'hard': 'Django', 'score': 0.913},
 {'easy': 'Python', 'hard': 'pyenv', 'score': 0.909},
 {'easy': 'Python', 'hard': 'Flask', 'score': 0.893},
 {'easy': 'Python', 'hard': '自然言語処理', 'score': 0.808},
 {'easy': 'Python', 'hard': 'Keras', 'score': 0.798},
 {'easy': 'Python', 'hard': 'Chainer', 'score': 0.781},
 {'easy': 'Python', 'hard': 'Kaggle', 'score': 0.75},
 {'easy': 'Python', 'hard': 'Pycharm', 'score': 0.71},
 {'easy': 'Python', 'hard': 'OpenCV', 'score': 0.704},
 {'easy': 'Python', 'hard': '機械学習', 'score': 0.688},
 {'easy': 'Python', 'hard': 'PyTorch', 'score': 0.684},
 {'easy':

# Pythonの上のノードについて調べる

In [19]:
for r in sorted(res, key=lambda x:x['score'], reverse=True):
    # 開始場所
    start = 0
    s = start
    for i, tag in enumerate(tags[start:len(tags)]):
        print("["+str(s+i)+"]:", tag)
        w1 = r["hard"]
        w2 = tag

        try:
            res = comparisonCheck(w1, w2)
            start += 1
        except:
            break

        if res:
            results.append(str(res))
        print()

    with open("easy_hard.csv", "w") as f:
        f.write("\n".join(results))
    print(res)
    print(start)

[0]: Groovy
numpy score: 0
Groovy score: 0
numpy and Groovy are irrelevant

[1]: mastodon
numpy score: 0
mastodon score: 0
numpy and mastodon are irrelevant

[2]: MacOSX
numpy score: 0
MacOSX score: 0
numpy and MacOSX are irrelevant

[3]: Bootstrap
numpy score: 0
Bootstrap score: 0
numpy and Bootstrap are irrelevant

[4]: RPA
numpy score: 0
RPA score: 0
numpy and RPA are irrelevant

[5]: GitHub
numpy score: 0
GitHub score: 0
numpy and GitHub are irrelevant

[6]: 初心者
numpy score: 0
初心者 score: 0
numpy and 初心者 are irrelevant

[7]: DDD
numpy score: 0
DDD score: 0
numpy and DDD are irrelevant

[8]: プログラミング
numpy score: 0
プログラミング score: 0
numpy and プログラミング are irrelevant

[9]: FreeBSD
numpy score: 0
FreeBSD score: 0
numpy and FreeBSD are irrelevant

[10]: postfix
numpy score: 0
postfix score: 0
numpy and postfix are irrelevant

[11]: Yum
numpy score: 0
Yum score: 0
numpy and Yum are irrelevant

[12]: ネットワーク
numpy score: 0
ネットワーク score: 0
numpy and ネットワーク are irrelevant

[13]: JavaScript
nump

numpy score: 0
ExcelVBA score: 0
numpy and ExcelVBA are irrelevant

[108]: Elm
numpy score: 0
Elm score: 0
numpy and Elm are irrelevant

[109]: RubyMine
numpy score: 0
RubyMine score: 0
numpy and RubyMine are irrelevant

[110]: pip
numpy score: 0
pip score: 0
numpy and pip are irrelevant

[111]: twilio
numpy score: 0
twilio score: 0
numpy and twilio are irrelevant

[112]: VisualStudioCode
numpy score: 0
VisualStudioCode score: 0
numpy and VisualStudioCode are irrelevant

[113]: Chrome
numpy score: 0
Chrome score: 0
numpy and Chrome are irrelevant

[114]: Laravel
numpy score: 0
Laravel score: 0
numpy and Laravel are irrelevant

[115]: 深層学習
numpy score: 0
深層学習 score: 0
numpy and 深層学習 are irrelevant

[116]: Linux
numpy score: 0
Linux score: 0
numpy and Linux are irrelevant

[117]: MachineLearning
numpy score: 0
MachineLearning score: 0
numpy and MachineLearning are irrelevant

[118]: Ethereum
numpy score: 0
Ethereum score: 0
numpy and Ethereum are irrelevant

[119]: PlayFramework
numpy sc

numpy score: 0
ATOM score: 0
numpy and ATOM are irrelevant

[212]: Subversion
numpy score: 0
Subversion score: 0
numpy and Subversion are irrelevant

[213]: chef
numpy score: 0
chef score: 0
numpy and chef are irrelevant

[214]: raspbian
numpy score: 0
raspbian score: 0
numpy and raspbian are irrelevant

[215]: bot
numpy score: 0
bot score: 0
numpy and bot are irrelevant

[216]: cli
numpy score: 0
cli score: 0
numpy and cli are irrelevant

[217]: Unity3D
numpy score: 0
Unity3D score: 0
numpy and Unity3D are irrelevant

[218]: Vagrant
numpy score: 0
Vagrant score: 0
numpy and Vagrant are irrelevant

[219]: PowerShell
numpy score: 0
PowerShell score: 0
numpy and PowerShell are irrelevant

[220]: PowerBI
numpy score: 0
PowerBI score: 0
numpy and PowerBI are irrelevant

[221]: animation
numpy score: 0
animation score: 0
numpy and animation are irrelevant

[222]: DeepLearning
numpy score: 0
DeepLearning score: 0
numpy and DeepLearning are irrelevant

[223]: FizzBuzz
numpy score: 0
FizzBuzz 

numpy score: 0
ASP.NET score: 0
numpy and ASP.NET are irrelevant

[316]: ポエム
numpy score: 0
ポエム score: 0
numpy and ポエム are irrelevant

[317]: LINEmessagingAPI
numpy score: 0
LINEmessagingAPI score: 0
numpy and LINEmessagingAPI are irrelevant

[318]: analytics
numpy score: 0
analytics score: 0
numpy and analytics are irrelevant

[319]: zabbix
numpy score: 0
zabbix score: 0
numpy and zabbix are irrelevant

[320]: openFrameworks
numpy score: 0
openFrameworks score: 0
numpy and openFrameworks are irrelevant

[321]: mqtt
numpy score: 0
mqtt score: 0
numpy and mqtt are irrelevant

[322]: RStudio
numpy score: 0
RStudio score: 0
numpy and RStudio are irrelevant

[323]: CTF
numpy score: 0
CTF score: 0
numpy and CTF are irrelevant

[324]: Blender
numpy score: 0
Blender score: 0
numpy and Blender are irrelevant

[325]: UI
numpy score: 0
UI score: 0
numpy and UI are irrelevant

[326]: スクレイピング
numpy score: 0
スクレイピング score: 0
numpy and スクレイピング are irrelevant

[327]: 備忘録
numpy score: 0
備忘録 score: 0
n

numpy score: 0
HoloLens score: 0
numpy and HoloLens are irrelevant

[420]: テスト
numpy score: 0
テスト score: 0
numpy and テスト are irrelevant

[421]: DynamoDB
numpy score: 0
DynamoDB score: 0
numpy and DynamoDB are irrelevant

[422]: 数値計算
numpy score: 0
数値計算 score: 0
numpy and 数値計算 are irrelevant

[423]: d3.js
numpy score: 0
d3.js score: 0
numpy and d3.js are irrelevant

[424]: OpenCV
numpy score: 0
OpenCV score: 0
numpy and OpenCV are irrelevant

[425]: Swift
numpy score: 0
Swift score: 0
numpy and Swift are irrelevant

[426]: ifttt
numpy score: 0
ifttt score: 0
numpy and ifttt are irrelevant

[427]: aws-cli
numpy score: 0
aws-cli score: 0
numpy and aws-cli are irrelevant

[428]: エンジニア
numpy score: 0
エンジニア score: 0
numpy and エンジニア are irrelevant

[429]: .NETFramework
numpy score: 0
.NETFramework score: 0
numpy and .NETFramework are irrelevant

[430]: jq
numpy score: 0
jq score: 0
numpy and jq are irrelevant

[431]: Bitbucket
numpy score: 0
Bitbucket score: 0
numpy and Bitbucket are irreleva

...

DeepLearning score: 0
awk score: 0
DeepLearning and awk are irrelevant

[72]: jsdo
DeepLearning score: 0
jsdo score: 0
DeepLearning and jsdo are irrelevant

[73]: 機械学習
DeepLearning score: 0.665
機械学習 score: 0.404
>> 機械学習 is easy!

[74]: LINQ
DeepLearning score: 0
LINQ score: 0
DeepLearning and LINQ are irrelevant

[75]: dockerfile
DeepLearning score: 0
dockerfile score: 0
DeepLearning and dockerfile are irrelevant

[76]: GIS
DeepLearning score: 0
GIS score: 0
DeepLearning and GIS are irrelevant

[77]: RxSwift
DeepLearning score: 0
RxSwift score: 0
DeepLearning and RxSwift are irrelevant

[78]: firefox
DeepLearning score: 0
firefox score: 0
DeepLearning and firefox are irrelevant

[79]: C#
DeepLearning score: 0
C# score: 0
DeepLearning and C# are irrelevant

[80]: Cygwin
DeepLearning score: 0
Cygwin score: 0
DeepLearning and Cygwin are irrelevant

[81]: データサイエンス
DeepLearning score: 0
データサイエンス score: 0
DeepLearning and データサイエンス are irrelevant

[82]: IAM
DeepLearning score: 0
IAM score: 0


DeepLearning score: 0
Erlang score: 0
DeepLearning and Erlang are irrelevant

[162]: swift
DeepLearning score: 0
swift score: 0
DeepLearning and swift are irrelevant

[163]: SPA
DeepLearning score: 0
SPA score: 0
DeepLearning and SPA are irrelevant

[164]: numpy
DeepLearning score: 0
numpy score: 0
DeepLearning and numpy are irrelevant

[165]: Kibana
DeepLearning score: 0
Kibana score: 0
DeepLearning and Kibana are irrelevant

[166]: GraphQL
DeepLearning score: 0
GraphQL score: 0
DeepLearning and GraphQL are irrelevant

[167]: python
DeepLearning score: 0.603
python score: 0
>> python is easy!

[168]: SQLite
DeepLearning score: 0
SQLite score: 0
DeepLearning and SQLite are irrelevant

[169]: jQuery
DeepLearning score: 0
jQuery score: 0
DeepLearning and jQuery are irrelevant

[170]: アルゴリズム
DeepLearning score: 0
アルゴリズム score: 0
DeepLearning and アルゴリズム are irrelevant

[171]: cron
DeepLearning score: 0
cron score: 0
DeepLearning and cron are irrelevant

[172]: Salesforce
DeepLearning score

DeepLearning score: 0
devops score: 0
DeepLearning and devops are irrelevant

[250]: ffmpeg
DeepLearning score: 0
ffmpeg score: 0
DeepLearning and ffmpeg are irrelevant

[251]: Elasticsearch
DeepLearning score: 0
Elasticsearch score: 0
DeepLearning and Elasticsearch are irrelevant

[252]: WebAPI
DeepLearning score: 0
WebAPI score: 0
DeepLearning and WebAPI are irrelevant

[253]: Julia
DeepLearning score: 0
Julia score: 0
DeepLearning and Julia are irrelevant

[254]: Titanium
DeepLearning score: 0
Titanium score: 0
DeepLearning and Titanium are irrelevant

[255]: PWA
DeepLearning score: 0
PWA score: 0
DeepLearning and PWA are irrelevant

[256]: solidity
DeepLearning score: 0
solidity score: 0
DeepLearning and solidity are irrelevant

[257]: CI
DeepLearning score: 0
CI score: 0
DeepLearning and CI are irrelevant

[258]: CentOS6.x
DeepLearning score: 0
CentOS6.x score: 0
DeepLearning and CentOS6.x are irrelevant

[259]: 環境構築
DeepLearning score: 0
環境構築 score: 0
DeepLearning and 環境構築 are ir

DeepLearning score: 0
Bash score: 0
DeepLearning and Bash are irrelevant

[338]: Qt
DeepLearning score: 0
Qt score: 0
DeepLearning and Qt are irrelevant

[339]: Scala
DeepLearning score: 0
Scala score: 0
DeepLearning and Scala are irrelevant

[340]: google
DeepLearning score: 0
google score: 0
DeepLearning and google are irrelevant

[341]: VPN
DeepLearning score: 0
VPN score: 0
DeepLearning and VPN are irrelevant

[342]: ECS
DeepLearning score: 0
ECS score: 0
DeepLearning and ECS are irrelevant

[343]: インストール
DeepLearning score: 0
インストール score: 0
DeepLearning and インストール are irrelevant

[344]: Eclipse
DeepLearning score: 0
Eclipse score: 0
DeepLearning and Eclipse are irrelevant

[345]: PHPUnit
DeepLearning score: 0
PHPUnit score: 0
DeepLearning and PHPUnit are irrelevant

[346]: Electron
DeepLearning score: 0
Electron score: 0
DeepLearning and Electron are irrelevant

[347]: Watson
DeepLearning score: 0
Watson score: 0
DeepLearning and Watson are irrelevant

[348]: ESLint
DeepLearning 

DeepLearning score: 0
ifttt score: 0
DeepLearning and ifttt are irrelevant

[427]: aws-cli
DeepLearning score: 0
aws-cli score: 0
DeepLearning and aws-cli are irrelevant

[428]: エンジニア
DeepLearning score: 0
エンジニア score: 0
DeepLearning and エンジニア are irrelevant

[429]: .NETFramework
DeepLearning score: 0
.NETFramework score: 0
DeepLearning and .NETFramework are irrelevant

[430]: jq
DeepLearning score: 0
jq score: 0
DeepLearning and jq are irrelevant

[431]: Bitbucket
DeepLearning score: 0
Bitbucket score: 0
DeepLearning and Bitbucket are irrelevant

[432]: ライブラリ
DeepLearning score: 0
ライブラリ score: 0
DeepLearning and ライブラリ are irrelevant

[433]: Python
DeepLearning score: 0.603
Python score: 0
>> Python is easy!

[434]: GitLab
DeepLearning score: 0
GitLab score: 0
DeepLearning and GitLab are irrelevant

[435]: 入門
DeepLearning score: 0
入門 score: 0
DeepLearning and 入門 are irrelevant

[436]: Flutter
DeepLearning score: 0
Flutter score: 0
DeepLearning and Flutter are irrelevant

[437]: Apache


In [138]:
with open("easy_hard.csv", "w") as f:
    f.write("\n".join(results))

In [24]:
res = [eval(r) for r in results]
sorted(res, key=lambda x:x['score'], reverse=True)

[{'easy': 'Python', 'hard': 'numpy', 'score': 1.0},
 {'easy': 'Python', 'hard': 'matplotlib', 'score': 1.0},
 {'easy': 'python', 'hard': 'numpy', 'score': 1.0},
 {'easy': 'Python', 'hard': 'numpy', 'score': 1.0},
 {'easy': 'python', 'hard': 'matplotlib', 'score': 1.0},
 {'easy': 'Python', 'hard': 'matplotlib', 'score': 1.0},
 {'easy': 'Python', 'hard': 'scikit-learn', 'score': 0.983},
 {'easy': 'python', 'hard': 'scikit-learn', 'score': 0.983},
 {'easy': 'Python', 'hard': 'scikit-learn', 'score': 0.983},
 {'easy': 'Python', 'hard': 'pandas', 'score': 0.949},
 {'easy': 'Python', 'hard': 'Jupyter', 'score': 0.949},
 {'easy': 'python', 'hard': 'pandas', 'score': 0.949},
 {'easy': 'Python', 'hard': 'pandas', 'score': 0.949},
 {'easy': 'python', 'hard': 'Jupyter', 'score': 0.949},
 {'easy': 'Python', 'hard': 'Jupyter', 'score': 0.949},
 {'easy': 'Python', 'hard': 'pip', 'score': 0.942},
 {'easy': 'Python', 'hard': 'Anaconda', 'score': 0.942},
 {'easy': 'python', 'hard': 'pip', 'score': 0.94

In [48]:
l = list(set([r["hard"].lower() for r in sorted(res, key=lambda x:x['score'], reverse=True)]))

In [50]:
l, len(l)

(['scikit-learn',
  'nlp',
  'pandas',
  'django',
  '論文読み',
  'ニューラルネットワーク',
  'anaconda',
  'kaggle',
  'mecab',
  '画像認識',
  'pyenv',
  'pycharm',
  'ai',
  'numpy',
  '人工知能',
  '自然言語処理',
  'ディープラーニング',
  'pip',
  'opencv',
  'deeplearning',
  'jupyter',
  'keras',
  'flask',
  '強化学習',
  '深層学習',
  '統計学',
  'pytorch',
  'cnn',
  'chainer',
  'machinelearning',
  '機械学習',
  'matplotlib'],
 32)

In [57]:
for i in results:
    
    tmp = eval(i)["easy"].lower()
    if tmp not in l:
        print(tmp)
        l.append(tmp)
        
    tmp = eval(i)["hard"].lower()
    if tmp not in l:
        print(tmp)
        l.append(tmp)

python


## 共通部分

In [90]:
len(set([eval(r)["easy"].lower() for r in results] + [eval(r)["hard"].lower() for r in results])), len(l)

(33, 33)

In [85]:
set([eval(r)["easy"].lower() for r in results] + [eval(r)["hard"].lower() for r in results]) & set(l)

{'ai',
 'anaconda',
 'chainer',
 'cnn',
 'deeplearning',
 'django',
 'flask',
 'jupyter',
 'kaggle',
 'keras',
 'machinelearning',
 'matplotlib',
 'mecab',
 'nlp',
 'numpy',
 'opencv',
 'pandas',
 'pip',
 'pycharm',
 'pyenv',
 'python',
 'pytorch',
 'scikit-learn',
 'ディープラーニング',
 'ニューラルネットワーク',
 '人工知能',
 '強化学習',
 '機械学習',
 '深層学習',
 '画像認識',
 '統計学',
 '自然言語処理',
 '論文読み'}

## 共通しない部分

In [86]:
set([eval(r)["easy"].lower() for r in results] + [eval(r)["hard"].lower() for r in results]) ^ set(l)

set()

# フォーマットしてcsvに書き込み

## フォーマット前のcsv読み込み

In [175]:
with open("easy_hard.csv", "r") as f:
    res = [eval(i) for i in f.read().split("\n")]

## 小文字に統一して重複を除く

In [196]:
new_res = []
for i in range(len(res)):
    if len(set([res[i]["easy"].lower()==nr["easy"] and res[i]["hard"].lower()==nr["hard"] for nr in new_res])) < 2:
        new_res.append({"easy": res[i]["easy"].lower(), "hard": res[i]["hard"].lower(), "score": res[i]["score"]})
        
new_res, len(new_res)

([{'easy': 'python', 'hard': 'chainer', 'score': 0.781},
  {'easy': 'python', 'hard': 'flask', 'score': 0.893},
  {'easy': 'python', 'hard': '機械学習', 'score': 0.688},
  {'easy': 'python', 'hard': '自然言語処理', 'score': 0.808},
  {'easy': 'python', 'hard': 'pip', 'score': 0.942},
  {'easy': 'python', 'hard': 'mecab', 'score': 0.661},
  {'easy': 'python', 'hard': 'numpy', 'score': 1.0},
  {'easy': 'python', 'hard': 'anaconda', 'score': 0.942},
  {'easy': 'python', 'hard': 'pandas', 'score': 0.949},
  {'easy': 'python', 'hard': 'deeplearning', 'score': 0.603},
  {'easy': 'python', 'hard': '統計学', 'score': 0.651},
  {'easy': 'python', 'hard': 'scikit-learn', 'score': 0.983},
  {'easy': 'python', 'hard': 'jupyter', 'score': 0.949},
  {'easy': 'python', 'hard': 'pyenv', 'score': 0.909},
  {'easy': 'python', 'hard': 'keras', 'score': 0.798},
  {'easy': 'python', 'hard': 'pycharm', 'score': 0.71},
  {'easy': 'python', 'hard': 'matplotlib', 'score': 1.0},
  {'easy': 'python', 'hard': 'ディープラーニング', 'sc

In [197]:
with open("format_easy_hard.csv", "w") as f:
    f.write("\n".join(map(str, new_res)))